<a href="https://colab.research.google.com/github/BerryWell/claim-adjuster-matching-algorithm/blob/main/%EC%86%90%ED%95%B4%EC%82%AC%EC%A0%95%EC%82%AC_%EB%B6%84%EB%A5%98_%EC%8B%9C%EC%8A%A4%ED%85%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 배경
* 기술의 발전으로 사회가 복잡해지고 다양한 보험 상품이 출시됨
* 이에 따른 보험금 청구 건수도 증가하고 관련 분쟁도 빈발하는 추세임
* 보험금 청구 시 보험소비자는 독립 손해사정사를 선임할 권리가 있음
* 하지만 관련 제도 홍보가 미비하여 이를 알고 있는 보험소비자도 적고 방법도 알고 있는 사람도 적음
* 따라서 본 알고리즘을 개발하여 보험소비자가 상황에 적합한 손해사정사를 매칭받고, 향후 타당성을 검증해보고자 함.

# 개요
* 본 프로그램은 아래 동작을 수행한다.
* 1. 보험사고 내용을 자연어(일상용어)로 입력한다.
* 2. 인공지능(AI) 기반으로 보험업법 시행규칙에 따른 손해사정사(신체, 재물, 차량)로 분류한다.
* 3. 보험소비자는 본인의 상황에 맞는 손해사정사의 분류를 안다.
* 추후 계획
* 1. 이 가능성을 바탕으로 향후 손해사정사 전문 분야 분류를 세분한다.
* 2. 각 손해사정사의 이력, 상세 프로필, 소개를 토대로 모델을 학습하도록 할 예정이다.
* 3. 손해사정사의 도움이 필요한 보험소비자가 적절한 손해사정사를 매칭받을 수 있도록 확장하여 손해사정사의 지위 향상과 보험소비자의 권익 향상을 하고자 한다.

# 사용 알고리즘
* 텍스트 전처리: konlp, Okt
* 분류: tf-idf

# 사용법
* 아래 내용을 셀별로 실행한다.

In [62]:
# @title 셀 왼쪽에 있는 플레이 버튼을 클릭하여 셀을 먼저 실행하세요. "모델 생성이 완료되었습니다" 메시지가 표시되면 아래 셀에 있는 폼에 내용을 입력하여 테스트해주세요.
!pip install pandas scikit-learn konlpy

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from konlpy.tag import Okt

# 1. 형태소 분석기 정의
okt = Okt()

def tokenize(text):
    return okt.morphs(text, stem=True)

# 손해사정사 분류: 신체, 재물, 차량

# 2. 예시 데이터 (3개 카테고리)
# 20개 데이터
data = {
    "text": [
        "어제 저녁 집에서 회사로 가는 길에 넘어져서 도움이 필요합니다.",
        "위암 진단을 받았는데 보험금을 어떻게 청구해야할지 몰라요.",
        "실손보험 청구했는데 보험금 지급 거절당했어요.",
        "뇌출혈로 쓰러진 형제가 있는데 보험금을 어떻게 청구해야할까요?",
        "간병보험의 청구 절차가 궁금해요.",
        "며칠 전 음식점에서 밥을 먹다가 돌을 씹어서 이가 부러졌어요. 어떻게 대처해야하나요?",
        "간경화가 있는데 간암으로 발전되었어요. 보험금을 제대로 받을 수 있나요?",
        "최근에 복강경 수술을 받아서 보험금 청구가 필요해요.",
        "길을 가다가 실수로 다른 사람의 휴대폰을 부수었어요.",
        "저희 집 강아지가 다른 사람 지갑을 물어뜯었어요.",
        "집에 불이 났는데 어떻게 보상받을 수 있나요?",
        "옆에 있는 하천이 범람해서 저희 집이 침수되었어요.",
        "집에 있던 노트북이 폭발해서 옆에 있는 귀중품에도 피해를 입었어요.",
        "운영하는 공장에 있는 기계가 갑자기 멈추었어요. 보험금 청구를 하고 싶어요.",
        "운전하고 가다가 실수로 길가에 있는 변압기를 쳐서 고장냈어요.",
        "어제 집앞 교차로에서 직진하다가 불법유턴하는 차랑 사고가 났어요.",
        "고속도로에 진입하다가 트럭과 사고가 났어요.",
        "횡단보도에서 사람을 실수로 치었어요. 어떻게 대처해야하나요?",
        "버스랑 사고가 났어요.",
        "신호위반으로 사고를 냈어요.",
        "중앙선침범 차량과 사고가 났어요.",
        "음주운전차와 사고가 났어요."
    ],
    "label": [
        "신체",
        "신체",
        "신체",
        "신체",
        "신체",
        "신체",
        "신체",
        "신체",
        "신체",
        "재물",
        "재물",
        "재물",
        "재물",
        "재물",
        "재물",
        "차량",
        "차량",
        "차량",
        "차량",
        "차량",
        "차량",
        "차량"
    ]
}

# 3. 데이터프레임 생성
df = pd.DataFrame(data)

# 4. 학습/테스트 분할
X_train, X_test, y_train, y_test = train_test_split(df["text"], df["label"], test_size=0.1, random_state=42)

# 5. 벡터라이저 + 모델 파이프라인
model = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=tokenize, max_features=500)),
    ('clf', LogisticRegression(max_iter=1000))
])

# 6. 학습
model.fit(X_train, y_train)

# 7. 평가
y_pred = model.predict(X_test)
print("분류 성능:\n")
print(classification_report(y_test, y_pred))

# 8. 예측 함수
def classify_korean_text(text):
    pred = model.predict([text])[0]
    return pred

print('모델 생성이 완료되었습니다!!!')

분류 성능:

              precision    recall  f1-score   support

          신체       0.50      0.50      0.50         2
          재물       0.00      0.00      0.00         1

    accuracy                           0.33         3
   macro avg       0.25      0.25      0.25         3
weighted avg       0.33      0.33      0.33         3

모델 생성이 완료되었습니다!!!


/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [66]:
# @title Default title text
example = "공장 가동이 멈추었어요." # @param {"type":"string","placeholder":"도움이 필요한 내용을 입력해주세요.(예. 항암 치료비를 청구해야해요. 운전하다가 사고가 났어요. 등)"}
#example = "암치료해야해요."
# 예시 테스트
print(f"\n입력 문장: '{example}'")
print(f"분류 결과: {classify_korean_text(example)}")


입력 문장: '공장 가동이 멈추었어요.'
분류 결과: 재물
